# M5 — Trabajo con el repositorio oficial `microsoft/Olive` y flujo “desde el código fuente” (VS Code + Jupyter)

## Objetivo
1. **Clonar** el repositorio oficial de Olive en tu máquina (desde VS Code o con `git clone`).
2. Crear un entorno reproducible (`.venv`) y seleccionar el **kernel** correcto en VS Code.
3. Instalar Olive **desde el repo** (modo editable) para poder leer/depurar el código.
4. Ejecutar una comprobación rápida (CLI) y preparar el terreno para usar notebooks/recetas oficiales.

## Prerrequisitos
- VS Code con soporte de Git (Source Control) y extensión de Jupyter.
- Python instalado.
- Acceso a `git` (o usar el comando integrado “Git: Clone” de VS Code).
- Conexión a Internet (para clonar/instalar desde GitHub).

## Fuentes oficiales usadas en este módulo
- VS Code: clonar repositorios con **Git: Clone** desde la Command Palette.  
- Olive: instalación desde fuente (`pip install git+...`) y **editable install** (`git clone ...` + `pip install -e .`).  
- VS Code: gestión/selección de kernels en notebooks (“Select Kernel”).  


In [ ]:
# Celda 1 — Comprobar entorno (desde el kernel actual)
import sys, subprocess
from pathlib import Path

print("Python executable:", sys.executable)
print("Python version:", sys.version)
print("CWD:", Path.cwd())

# ¿Git disponible?
git = subprocess.run(["git", "--version"], text=True, capture_output=True)
print("\nGit returncode:", git.returncode)
print("Git stdout:", git.stdout.strip())
print("Git stderr:", git.stderr.strip())

# Olive instalado en este entorno (si existe)
print("\n--- pip show olive-ai ---")
subprocess.run([sys.executable, "-m", "pip", "show", "olive-ai"], check=False)


## Explicación breve

En M0–M4 trabajamos con Olive como paquete instalado. En M5 damos el salto a:
- **Repos oficial**: para ver notebooks, estructura del proyecto y depurar.
- **Instalación editable**: para que los cambios en el repo se reflejen al importar `olive`.

Esto te permite:
- entender mejor errores de config/registro de passes
- seguir ejemplos del propio repositorio
- preparar contribuciones (si lo deseas)

> Importante: en VS Code, el kernel del notebook debe apuntar al `.venv` correcto, usando el selector **Select Kernel**.


## 1) Clonar el repositorio (opción A: VS Code)

En VS Code:
1. Abre la Command Palette (`Ctrl+Shift+P`)
2. Ejecuta **Git: Clone**
3. Pega la URL del repositorio (por ejemplo, `https://github.com/microsoft/Olive`)
4. Elige la carpeta donde guardarlo y abre el repositorio

> Si prefieres terminal, usa la opción B (siguiente sección).


## 2) Clonar el repositorio (opción B: desde el notebook/terminal)

La guía oficial de instalación editable de Olive indica:

```bash
git clone https://github.com/microsoft/Olive
cd Olive
pip install -e .
```

En esta práctica, clonaremos en una carpeta `external/Olive` dentro de tu proyecto (puedes cambiarla).


In [ ]:
# Celda 2 — (Opcional) clonar el repo desde aquí con git
import subprocess
from pathlib import Path

root = Path.cwd()
external_dir = root / "external"
repo_dir = external_dir / "Olive"

external_dir.mkdir(exist_ok=True)

if repo_dir.exists():
    print("Repo ya existe:", repo_dir.resolve())
else:
    cmd = ["git", "clone", "https://github.com/microsoft/Olive", str(repo_dir)]
    print("Running:", " ".join(cmd))
    r = subprocess.run(cmd, text=True)
    print("returncode:", r.returncode)

print("Repo path:", repo_dir.resolve())


## 3) Crear `.venv` dentro del repo clonado (recomendado)

En la documentación de Olive se recomienda instalar en un entorno virtual y luego instalar Olive.  
En Windows, la activación típica es `tutorial-env/Scripts/activate` (ver doc oficial de `venv`).

### Comandos (Terminal integrado de VS Code)
> Ejecuta estos comandos **dentro** de `external/Olive` (o donde hayas clonado).

**PowerShell**
```powershell
python -m venv .venv
.\.venv\Scripts\Activate.ps1
python -m pip install -U pip
python -m pip install -e .
```

**CMD**
```bat
python -m venv .venv
.\.venv\Scripts\activate.bat
python -m pip install -U pip
python -m pip install -e .
```

> Nota: Olive también documenta instalación desde fuente sin clonar usando `pip install git+https://github.com/microsoft/Olive`.


## 4) Seleccionar el kernel correcto en VS Code

En el notebook, abre el selector en la esquina superior derecha (**Select Kernel**) y elige el Python de:

`external/Olive/.venv`

Esto asegura que:
- `python -m olive ...` y `import olive` usen el entorno correcto
- no se “cuele” el Python global (lo que ya vimos en M2/M3)


In [ ]:
# Celda 3 — (Para ejecutar DESPUÉS de seleccionar el kernel del repo) verificación rápida
import sys, subprocess
from pathlib import Path

print("Python executable:", sys.executable)
print("Repo root guess:", Path.cwd())

print("\nOlive import + ubicación:")
import olive
print("olive.__file__:", getattr(olive, "__file__", None))

print("\nCLI help (olive -h) usando el python del kernel:")
r = subprocess.run([sys.executable, "-m", "olive", "-h"], text=True, capture_output=True)
print("returncode:", r.returncode)
print(r.stdout[:1500])
print(r.stderr[:500])


## 5) (Opcional) Probar un comando moderno: `olive optimize`

El README del repositorio muestra el flujo de **Automatic Optimizer** con `olive optimize` y flags como `--model_name_or_path`, `--precision` y `--output_path`.

⚠️ Este paso descarga modelos de Hugging Face y puede tardar / requerir recursos. Lo dejamos como opcional.


## Verificación
- Puedes clonar el repo (por VS Code o `git clone`).
- Dentro del repo, creas `.venv` y haces `pip install -e .`.
- En VS Code, el notebook usa el kernel correcto (el Python dentro de `.venv`).
- `python -m olive -h` funciona y muestra los subcomandos.

## Errores comunes y diagnóstico
1) **El notebook usa Python global**
   - `sys.executable` apunta a `C:\Users\...\Python313\python.exe` en vez de `...\.venv\Scripts\python.exe`.
   - Solución: Select Kernel → elige `.venv`.

2) **No tienes git en PATH**
   - `git --version` falla.
   - Solución: clona desde VS Code (Git: Clone) o instala Git.

3) **Permisos al activar en PowerShell**
   - Si `Activate.ps1` está bloqueado, revisa la doc de `venv` (política de ejecución).

## Siguiente paso (M6)
Proyecto final: pipeline completo con métricas y un mini‑reporte (modelo → optimización/cuantización → medición ORT → artefactos).
